# Settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [3]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


In [4]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# Dataset loading

In [5]:
dataset_name='dsprites'

In [6]:
import tensorflow as tf

In [7]:
inputs_shape= image_size=(64, 64, 1)
batch_size = 32
latents_dim = 32
intermediate_dim = 50

In [8]:
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='dsprites',
    version=0.1.0,
    description='dSprites is a dataset of 2D shapes procedurally generated from 6 ground truth
independent latent factors. These factors are *color*, *shape*, *scale*,
*rotation*, *x* and *y* positions of a sprite.

All possible combinations of these latents are present exactly once,
generating N = 737280 total images.

### Latent factor values

*   Color: white
*   Shape: square, ellipse, heart
*   Scale: 6 values linearly spaced in [0.5, 1]
*   Orientation: 40 values in [0, 2 pi]
*   Position X: 32 values in [0, 1]
*   Position Y: 32 values in [0, 1]

We varied one latent at a time (starting from Position Y, then Position X, etc),
and sequentially stored the images in fixed order.
Hence the order along the first dimension is fixed and allows you to map back to
the value of the latents corresponding to that image.

We chose the latents values deliberately to have the smallest step changes
while ensuring that all pixel outp

In [9]:
TRAIN_BUF = 600
TEST_BUF = 100

import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)


In [10]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'].numpy().max())
    break

In [11]:
_instance_scale

1.0

In [12]:
import numpy as np
from collections.abc import Iterable

In [13]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [14]:
_outputs_shape

4096

# Model's Layers definition

In [15]:
units=20
c=16
enc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=units*9, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=c*c*units, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(c , c, units)),
    tf.keras.layers.Conv2DTranspose(filters=units, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=units*3, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [16]:
model_name = dataset_name+'DIP_Bayesian_Covariance_AE_Convolutional_reconst_ellwlb'
experiments_dir='experiments'+sep_local+model_name

In [17]:
from training.disentangled_inferred_prior.bayesian.ByCovariance.Bayesian_Covariance_AE import Bayesian_Covariance_AE as AE

In [18]:
inputs_shape=image_size

In [19]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [20]:
from utils.data_and_files.file_utils import create_if_not_exist

In [21]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [22]:
create_if_not_exist(_restore)
_restore

'experiments\\dspritesDIP_Bayesian_Covariance_AE_Convolutional_reconst_ellwlb\\var_save_dir'

In [23]:
#to restore trained model, set filepath=_restore

In [24]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 31, 31, 20)        200       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 180)       32580     
_________________________________________________________________
flatten (Flatten)            (None, 40500)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1296032   
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5120)              168960    
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 20)        3620      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 60)        10860     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 1)         541       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 1)         4

  WARNING  | None


In [25]:
#ae.compile(metrics=None)
ae.compile()

Model: "dspritesDIP_Bayesian_Covariance_AE_Convolutional_reconst_ellwlb"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_inputs (InputLayer)   [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
inference (Model)               (None, 32)           1328940     inference_inputs[0][0]           
__________________________________________________________________________________________________
tf_op_layer_add (TensorFlowOpLa [(None, 32)]         0           inference[1][0]                  
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(None, 32, 1)]      0           inference[1][0]                  
____________________________________

# Callbacks

In [26]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [27]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [28]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [29]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\dspritesDIP_Bayesian_Covariance_AE_Convolutional_reconst_ellwlb\\csv_dir\\dspritesDIP_Bayesian_Covariance_AE_Convolutional_reconst_ellwlb.csv'

In [30]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [31]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [32]:
import numpy as np

In [33]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

The system cannot find the path specified.


In [34]:
from os.path import abspath

from data.gt_load.datasets import load
DATA_PATH = DATA_DOWN_PATH +sep_local+'.gt_datasets'
absolute = abspath(DATA_PATH)
print("DATA_PATH",absolute)


DATA_PATH C:\Users\Khalid\Documents\projects\Generative_Models\data\.gt_datasets


In [35]:
eval_dataset = load(dataset_name='dsprites_full', dataset_path=DATA_PATH)

In [36]:
from training.callbacks.disentangle_supervied import DisentanglementSuperviedMetrics
from training.callbacks.disentangle_unsupervied import DisentanglementUnsuperviedMetrics

In [37]:
gts_csv = os.path.join(experiments_dir, 'csv_dir', 'gts_metrics')
gtu_csv = os.path.join(experiments_dir, 'csv_dir', 'gtu_metrics')

In [38]:
gts_mertics = DisentanglementSuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gts_csv,
    num_train=10000,
    num_test=100,
    batch_size=batch_size,
    continuous_factors=False,
    gt_freq=10
)
gtu_mertics = DisentanglementUnsuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gtu_csv,
    num_train=10000,
    num_test=100,
    batch_size=batch_size,
    gt_freq=10
)

# Model Training

In [39]:
ae.fit(
    x=train_ds,
    input_kw='image',
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg, gts_mertics, gtu_mertics],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)

Train for 1000 steps, validate for 1000 steps
Epoch 1/1000000


  65/1000 [>.............................] - ETA: 1:18:59 - loss: 2845.8469 - bayesian_divergent_loss: 0.3905 - x_logits_loss: 2845.4561 - x_logits_psnr: 6.0086 - x_logits_ssim: 0.0201 - x_logits_sharpdiff: 10.3252 - x_logits_mean_absolute_error: 0.4997 - x_logits_mean_squared_error: 0.250 - ETA: 26:41 - loss: 1798.5385 - bayesian_divergent_loss: 0.3096 - x_logits_loss: 1798.2012 - x_logits_psnr: 8.8712 - x_logits_ssim: 0.0065 - x_logits_sharpdiff: 6.4880 - x_logits_mean_absolute_error: 0.2959 - x_logits_mean_squared_error: 0.1464  - ETA: 16:13 - loss: 1617.6095 - bayesian_divergent_loss: 0.2444 - x_logits_loss: 1617.3138 - x_logits_psnr: 9.4567 - x_logits_ssim: 0.0040 - x_logits_sharpdiff: 5.6083 - x_logits_mean_absolute_error: 0.2430 - x_logits_mean_squared_error: 0.12 - ETA: 11:44 - loss: 1544.1827 - bayesian_divergent_loss: 0.2133 - x_logits_loss: 1543.9055 - x_logits_psnr: 9.7059 - x_logits_ssim: 0.0028 - x_logits_sharpdiff: 5.2336 - x_logits_mean_absolute_error: 0.2197 - x_logits

 199/1000 [====>.........................] - ETA: 1:01 - loss: 1187.8653 - bayesian_divergent_loss: 4.2272 - x_logits_loss: 1183.5433 - x_logits_psnr: 10.5674 - x_logits_ssim: -0.0023 - x_logits_sharpdiff: 4.3236 - x_logits_mean_absolute_error: 0.1618 - x_logits_mean_squared_error: 0.089 - ETA: 1:01 - loss: 1183.7291 - bayesian_divergent_loss: 4.2779 - x_logits_loss: 1179.3564 - x_logits_psnr: 10.5761 - x_logits_ssim: -0.0024 - x_logits_sharpdiff: 4.3232 - x_logits_mean_absolute_error: 0.1616 - x_logits_mean_squared_error: 0.089 - ETA: 1:00 - loss: 1179.2289 - bayesian_divergent_loss: 4.3067 - x_logits_loss: 1174.8274 - x_logits_psnr: 10.5869 - x_logits_ssim: -0.0025 - x_logits_sharpdiff: 4.3231 - x_logits_mean_absolute_error: 0.1614 - x_logits_mean_squared_error: 0.088 - ETA: 59s - loss: 1175.4312 - bayesian_divergent_loss: 4.3267 - x_logits_loss: 1171.0096 - x_logits_psnr: 10.5948 - x_logits_ssim: -0.0025 - x_logits_sharpdiff: 4.3226 - x_logits_mean_absolute_error: 0.1613 - x_logits_

 267/1000 [=======>......................] - ETA: 46s - loss: 1086.8205 - bayesian_divergent_loss: 4.9129 - x_logits_loss: 1081.8115 - x_logits_psnr: 10.8003 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3168 - x_logits_mean_absolute_error: 0.1577 - x_logits_mean_squared_error: 0.08 - ETA: 46s - loss: 1084.8046 - bayesian_divergent_loss: 4.9197 - x_logits_loss: 1079.7888 - x_logits_psnr: 10.8052 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3166 - x_logits_mean_absolute_error: 0.1576 - x_logits_mean_squared_error: 0.08 - ETA: 46s - loss: 1082.8593 - bayesian_divergent_loss: 4.9201 - x_logits_loss: 1077.8430 - x_logits_psnr: 10.8098 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3166 - x_logits_mean_absolute_error: 0.1575 - x_logits_mean_squared_error: 0.08 - ETA: 46s - loss: 1081.0147 - bayesian_divergent_loss: 4.9293 - x_logits_loss: 1075.9893 - x_logits_psnr: 10.8140 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3164 - x_logits_mean_absolute_error: 0.1575 - x_logits_mean_s

 335/1000 [=========>....................] - ETA: 38s - loss: 1030.5302 - bayesian_divergent_loss: 5.0575 - x_logits_loss: 1025.3760 - x_logits_psnr: 10.9485 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3133 - x_logits_mean_absolute_error: 0.1552 - x_logits_mean_squared_error: 0.08 - ETA: 38s - loss: 1029.2497 - bayesian_divergent_loss: 5.0708 - x_logits_loss: 1024.0823 - x_logits_psnr: 10.9521 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3133 - x_logits_mean_absolute_error: 0.1551 - x_logits_mean_squared_error: 0.08 - ETA: 38s - loss: 1028.0169 - bayesian_divergent_loss: 5.0679 - x_logits_loss: 1022.8524 - x_logits_psnr: 10.9554 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3133 - x_logits_mean_absolute_error: 0.1551 - x_logits_mean_squared_error: 0.08 - ETA: 38s - loss: 1026.7091 - bayesian_divergent_loss: 5.0716 - x_logits_loss: 1021.5408 - x_logits_psnr: 10.9591 - x_logits_ssim: -0.0040 - x_logits_sharpdiff: 4.3132 - x_logits_mean_absolute_error: 0.1550 - x_logits_mean_s

...

 407/1000 [===========>..................] - ETA: 23s - loss: 774.4060 - bayesian_divergent_loss: 0.2241 - x_logits_loss: 773.9615 - x_logits_psnr: 11.7304 - x_logits_ssim: 7.3133e-04 - x_logits_sharpdiff: 4.2160 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 23s - loss: 774.4203 - bayesian_divergent_loss: 0.2243 - x_logits_loss: 773.9756 - x_logits_psnr: 11.7303 - x_logits_ssim: 7.3146e-04 - x_logits_sharpdiff: 4.2159 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 23s - loss: 774.4145 - bayesian_divergent_loss: 0.2243 - x_logits_loss: 773.9699 - x_logits_psnr: 11.7304 - x_logits_ssim: 7.3166e-04 - x_logits_sharpdiff: 4.2160 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 23s - loss: 774.4178 - bayesian_divergent_loss: 0.2243 - x_logits_loss: 773.9731 - x_logits_psnr: 11.7303 - x_logits_ssim: 7.3198e-04 - x_logits_sharpdiff: 4.2161 - x_logits_mean_absolute_error: 0.1345 - x_logits

 475/1000 [=============>................] - ETA: 20s - loss: 774.3080 - bayesian_divergent_loss: 0.2242 - x_logits_loss: 773.8632 - x_logits_psnr: 11.7307 - x_logits_ssim: 7.2170e-04 - x_logits_sharpdiff: 4.2163 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 20s - loss: 774.2607 - bayesian_divergent_loss: 0.2243 - x_logits_loss: 773.8159 - x_logits_psnr: 11.7309 - x_logits_ssim: 7.2184e-04 - x_logits_sharpdiff: 4.2163 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 20s - loss: 774.2163 - bayesian_divergent_loss: 0.2244 - x_logits_loss: 773.7714 - x_logits_psnr: 11.7311 - x_logits_ssim: 7.2451e-04 - x_logits_sharpdiff: 4.2163 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 20s - loss: 774.2392 - bayesian_divergent_loss: 0.2243 - x_logits_loss: 773.7944 - x_logits_psnr: 11.7310 - x_logits_ssim: 7.2633e-04 - x_logits_sharpdiff: 4.2163 - x_logits_mean_absolute_error: 0.1345 - x_logits

 543/1000 [===============>..............] - ETA: 18s - loss: 774.2214 - bayesian_divergent_loss: 0.2235 - x_logits_loss: 773.7775 - x_logits_psnr: 11.7311 - x_logits_ssim: 7.1974e-04 - x_logits_sharpdiff: 4.2161 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 18s - loss: 774.1807 - bayesian_divergent_loss: 0.2235 - x_logits_loss: 773.7368 - x_logits_psnr: 11.7314 - x_logits_ssim: 7.2062e-04 - x_logits_sharpdiff: 4.2162 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 18s - loss: 774.1830 - bayesian_divergent_loss: 0.2234 - x_logits_loss: 773.7390 - x_logits_psnr: 11.7313 - x_logits_ssim: 7.2064e-04 - x_logits_sharpdiff: 4.2162 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 18s - loss: 774.1673 - bayesian_divergent_loss: 0.2233 - x_logits_loss: 773.7234 - x_logits_psnr: 11.7314 - x_logits_ssim: 7.2098e-04 - x_logits_sharpdiff: 4.2162 - x_logits_mean_absolute_error: 0.1345 - x_logits

 553/1000 [===============>..............] - ETA: 16s - loss: 774.1365 - bayesian_divergent_loss: 0.2231 - x_logits_loss: 773.6926 - x_logits_psnr: 11.7314 - x_logits_ssim: 7.1533e-04 - x_logits_sharpdiff: 4.2163 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 16s - loss: 774.1319 - bayesian_divergent_loss: 0.2231 - x_logits_loss: 773.6880 - x_logits_psnr: 11.7314 - x_logits_ssim: 7.1475e-04 - x_logits_sharpdiff: 4.2163 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 16s - loss: 774.1624 - bayesian_divergent_loss: 0.2232 - x_logits_loss: 773.7184 - x_logits_psnr: 11.7313 - x_logits_ssim: 7.1382e-04 - x_logits_sharpdiff: 4.2162 - x_logits_mean_absolute_error: 0.1345 - x_logits_mean_squared_error: 0.067 - ETA: 15s - loss: 774.1627 - bayesian_divergent_loss: 0.2232 - x_logits_loss: 773.7187 - x_logits_psnr: 11.7313 - x_logits_ssim: 7.1246e-04 - x_logits_sharpdiff: 4.2162 - x_logits_mean_absolute_error: 0.1345 - x_logits

KeyError: 'val_bayesian_divergent_loss'

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)